# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 11:00AM,258451,10,9220883,"Citieffe, Inc.",Released
1,Mar 1 2023 11:00AM,258452,10,Enova-11430,Emerginnova,Released
2,Mar 1 2023 11:00AM,258452,10,Enova-11431,Emerginnova,Released
3,Mar 1 2023 10:57AM,258450,10,Clear-9220912,"ClearSpec, LLC",Released
4,Mar 1 2023 10:53AM,258449,10,9220910,"Zelenko Labs, LLC",Released
5,Mar 1 2023 10:53AM,258448,22,642659,"NBTY Global, Inc.",Released
6,Mar 1 2023 10:50AM,258447,10,Yusen-9220902,Yusen – 3D Matrix,Released
7,Mar 1 2023 10:50AM,258446,10,9220879,Bio-PRF,Released
8,Mar 1 2023 10:50AM,258446,10,PRF-54484,Bio-PRF,Released
9,Mar 1 2023 10:50AM,258446,10,PRF-54436,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,258448,Released,1
27,258449,Released,1
28,258450,Released,1
29,258451,Released,1
30,258452,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258448,NaN,NaN,1.0
258449,NaN,NaN,1.0
258450,NaN,NaN,1.0
258451,NaN,NaN,1.0
258452,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258268,0.0,45.0,5.0
258368,2.0,6.0,5.0
258375,0.0,0.0,56.0
258391,0.0,2.0,1.0
258397,2.0,4.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258268,0,45,5
258368,2,6,5
258375,0,0,56
258391,0,2,1
258397,2,4,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258268,0,45,5
1,258368,2,6,5
2,258375,0,0,56
3,258391,0,2,1
4,258397,2,4,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258268,,45,5
1,258368,2,6,5
2,258375,,,56
3,258391,,2,1
4,258397,2,4,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc."
1,Mar 1 2023 11:00AM,258452,10,Emerginnova
3,Mar 1 2023 10:57AM,258450,10,"ClearSpec, LLC"
4,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC"
5,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc."
6,Mar 1 2023 10:50AM,258447,10,Yusen – 3D Matrix
7,Mar 1 2023 10:50AM,258446,10,Bio-PRF
21,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc."
23,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC"
24,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",,,1
1,Mar 1 2023 11:00AM,258452,10,Emerginnova,,,2
2,Mar 1 2023 10:57AM,258450,10,"ClearSpec, LLC",,,1
3,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",,,1
4,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",,,1
5,Mar 1 2023 10:50AM,258447,10,Yusen – 3D Matrix,,,1
6,Mar 1 2023 10:50AM,258446,10,Bio-PRF,,,14
7,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",,,2
8,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",,,1
9,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",1,,
1,Mar 1 2023 11:00AM,258452,10,Emerginnova,2,,
2,Mar 1 2023 10:57AM,258450,10,"ClearSpec, LLC",1,,
3,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",1,,
4,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",1,,
5,Mar 1 2023 10:50AM,258447,10,Yusen – 3D Matrix,1,,
6,Mar 1 2023 10:50AM,258446,10,Bio-PRF,14,,
7,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",2,,
8,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",1,,
9,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",1,,
1,Mar 1 2023 11:00AM,258452,10,Emerginnova,2,,
2,Mar 1 2023 10:57AM,258450,10,"ClearSpec, LLC",1,,
3,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",1,,
4,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Mar 1 2023 11:00AM,258452,10,Emerginnova,2.0,NaN,NaN
2,Mar 1 2023 10:57AM,258450,10,"ClearSpec, LLC",1.0,NaN,NaN
3,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",1.0,NaN,NaN
4,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Mar 1 2023 11:00AM,258452,10,Emerginnova,2.0,0.0,0.0
2,Mar 1 2023 10:57AM,258450,10,"ClearSpec, LLC",1.0,0.0,0.0
3,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",1.0,0.0,0.0
4,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3876444,141.0,12.0,4.0
12,258421,1.0,0.0,0.0
15,1033492,64.0,45.0,0.0
16,258425,1.0,0.0,0.0
19,775299,4.0,0.0,0.0
22,516865,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3876444,141.0,12.0,4.0
1,12,258421,1.0,0.0,0.0
2,15,1033492,64.0,45.0,0.0
3,16,258425,1.0,0.0,0.0
4,19,775299,4.0,0.0,0.0
5,22,516865,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,141.0,12.0,4.0
1,12,1.0,0.0,0.0
2,15,64.0,45.0,0.0
3,16,1.0,0.0,0.0
4,19,4.0,0.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,141.0
1,12,Released,1.0
2,15,Released,64.0
3,16,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0
Executing,12.0,0.0,45.0,0.0,0.0,0.0
Released,141.0,1.0,64.0,1.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,45.0,0.0,0.0,0.0
2,Released,141.0,1.0,64.0,1.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,45.0,0.0,0.0,0.0
2,Released,141.0,1.0,64.0,1.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()